# Battery Check (a CRONTAB task)

This monitor script is used for motes using an internal backup battery.  The battery voltage is reported using the JSON object "batt". 

The following issues are used in tickets opened by this script:
- CODE_RED
- CODE_YELLOW
- CODE_GREEN


### CODE_RED BATTERY CRITICAL<br>
This issue can arise if a mote remains on the battery for an excessive period of time.
### CODE_YELLOW BATTERY DISCHARGING<br>
This issue will occur if the mote loses mains power for a moderate period of time.
### CODE_GREEN BATTERY OK<br>
This is the normal status for a battery in the charging state.

In [1]:
DEPLOYED_ACTIVE  = 10001

#Ticket Types
CODE_GREEN  = 11000  # normally not ticketed
CODE_RED    = 11005
CODE_YELLOW = 11006

In [2]:
import os
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms')
print("============================================")
print("/  Battery Check is running.                  /")
print("============================================")
import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser
import time

print(os.getcwd())
config = configparser.ConfigParser()
config.read("../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']
    
  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])
      
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

/  Battery Check is running.                  /
/home/sensei/jupy-notebooks/Analytics/PorterFarms
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [3]:
def ticketIssue(conn, node_id, ticket_type, description):
  openTicket = kt.ticketExists(conn, node_id, ticket_type, [kt._OPEN_STATUS, kt._WORKING_STATUS])
  if openTicket is None:
    query = "SELECT    node.name, node.location_id, location.description, location.slackalertchannel_id, \
                       location.imageurl, customer.slacktoken, slackchannel.idslackchannel, slackchannel.channelid \
                       FROM kanji_node node JOIN kanji_location location ON location.idlocation=node.location_id \
                       JOIN kanji_slackchannel slackchannel ON location.slackalertchannel_id=slackchannel.idslackchannel \
                       JOIN kanji_customer customer ON customer.idcustomer=node.customer_id \
                       WHERE node.idnode={}".format(node_id)
    logger(_LOG_INFO, query)
    df = pd.read_sql(query, conn)    
    locationid = df['location_id'][0]
    locationdescription = df['description'][0]
    alerttextquery = "SELECT alerttext FROM kanji_tickettype WHERE idtickettype={}".format(ticket_type)
    df2 = pd.read_sql(alerttextquery, conn)
    ticketdescription = description + " " + df2['alerttext'][0]
    slackchannelid = df['idslackchannel'][0]
    slackchannelname = df['channelid'][0]
    nodename = df['name'][0]
    mentions = "@Charlie"
    locationimageurl = df['imageurl'][0]
    slacktoken = df['slacktoken'][0]
    
    logger(_LOG_DEBUG,"channel={} token={}".format(slackchannelid,slacktoken))
    
    ticketid = kt.openticket(conn, node_id, locationid, ticketdescription, 2, 3, ticket_type, slackchannelid)
    ts = kt.slackticket(nodename, locationdescription, ticketdescription, mentions, 2, 3, locationimageurl, \
                        slacktoken,slackchannelname, ticketid, 0)
    kt.updateTicket(conn, ticketid, ts)  
    logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
  else:
    logger(_LOG_INFO, "There is an existing ticket {} for this issue. {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))

In [4]:
messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

In [5]:
def postMessageToSlack(text):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = "infrastructure"
    response = sc.api_call("chat.postMessage", channel=slackchannel, text=text, blocks=[])
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(response)
    else:
      print("Ok posting message to Slack channel")   

# Strategy
Query the eventlog table for all motes in the deployed state.

Examine the eventdata JSON object for the battery sensor object type.  If found, determine the status of the battery.

In [6]:
_LOG_LEVEL = _LOG_DEBUG

_BATTERY_DISCHARGING_THRESHOLD_VALUE = 3.8
_BATTERY_CRITICAL_THRESHOLD_VALUE = 3.6

motequery = "SELECT node.idnode, node.name, location.description as location FROM kanji_node node \
             JOIN kanji_location location ON location.idlocation=node.location_id \
             WHERE deploystate_id={}".format(DEPLOYED_ACTIVE)
logger(_LOG_DEBUG, "motequery={}".format(motequery))

motes = pd.read_sql(motequery, conn)
if motes.size>0:
 for moteind in motes.index:
  motename = motes['name'][moteind]
  moteid = motes['idnode'][moteind]
  motelocation = motes['location'][moteind]
  print("mote = {} {} at {}".format(motename, moteid, motelocation))
  eventquery = "SELECT timestamp, eventdata FROM kanji_eventlog \
                WHERE node_id={} AND \
                timestamp > now() - INTERVAL '{} minutes' \
                ORDER BY timestamp desc LIMIT 1".format(moteid, 5)
  events = pd.read_sql(eventquery, conn)
  if events.size>0:
    for index in events.index:
      timestamp = events['timestamp'][index]
      eventdata = events['eventdata'][index]
      eventdata = eventdata.replace("'","\"")
      #print(eventdata)
      jsondata = json.loads(eventdata)
      try: 
        battery_voltage = float(jsondata["batt"])
        if (battery_voltage > _BATTERY_DISCHARGING_THRESHOLD_VALUE):
          print("battery is nominal {}volts at {}".format(battery_voltage, timestamp))
        elif (battery_voltage > _BATTERY_CRITICAL_THRESHOLD_VALUE):
          print("battery is discharging {}volts at {}".format(battery_voltage, timestamp))
          ticketIssue(conn, moteid, CODE_RED, "CODE_YELLOW Battery Discharging.")
          postMessageToSlack("CODE_YELLOW Battery Discharging. {}".format(motelocation))
        else:
          print("battery is criticial {}volts at {}".format(battery_voltage, timestamp))
          ticketIssue(conn, moteid, CODE_RED, "CODE_RED Battery Critical.")
          postMessageToSlack("CODE_RED Battery Critical. {}".format(motelocation))          
      except KeyError:
          pass
  else:
    print("Query {} returned no results".format(eventquery))
        
  
            

motequery=SELECT node.idnode, node.name, location.description as location FROM kanji_node node              JOIN kanji_location location ON location.idlocation=node.location_id              WHERE deploystate_id=10001
mote = agMote-20010 20010 at Breeding
battery is nominal 4.02volts at 2022-06-10 14:30:26.967922+00:00
mote = agMote-20015 20015 at Breeding
Query SELECT timestamp, eventdata FROM kanji_eventlog                 WHERE node_id=20015 AND                 timestamp > now() - INTERVAL '5 minutes'                 ORDER BY timestamp desc LIMIT 1 returned no results
mote = agMote-20000 20000 at Mobile Unit
battery is nominal 6.59volts at 2022-06-10 14:30:33.873032+00:00
